# Lambda School Data Science Module 143

## Introduction to Bayesian Inference

!['Detector! What would the Bayesian statistician say if I asked him whether the--' [roll] 'I AM A NEUTRINO DETECTOR, NOT A LABYRINTH GUARD. SERIOUSLY, DID YOUR BRAIN FALL OUT?' [roll] '... yes.'](https://imgs.xkcd.com/comics/frequentists_vs_bayesians_2x.png)

*[XKCD 1132](https://www.xkcd.com/1132/)*


## Prepare - Bayes' Theorem and the Bayesian mindset

Bayes' theorem possesses a near-mythical quality - a bit of math that somehow magically evaluates a situation. But this mythicalness has more to do with its reputation and advanced applications than the actual core of it - deriving it is actually remarkably straightforward.

### The Law of Total Probability

By definition, the total probability of all outcomes (events) if some variable (event space) $A$ is 1. That is:

$$P(A) = \sum_n P(A_n) = 1$$

The law of total probability takes this further, considering two variables ($A$ and $B$) and relating their marginal probabilities (their likelihoods considered independently, without reference to one another) and their conditional probabilities (their likelihoods considered jointly). A marginal probability is simply notated as e.g. $P(A)$, while a conditional probability is notated $P(A|B)$, which reads "probability of $A$ *given* $B$".

The law of total probability states:

$$P(A) = \sum_n P(A | B_n) P(B_n)$$

In words - the total probability of $A$ is equal to the sum of the conditional probability of $A$ on any given event $B_n$ times the probability of that event $B_n$, and summed over all possible events in $B$.

### The Law of Conditional Probability

What's the probability of something conditioned on something else? To determine this we have to go back to set theory and think about the intersection of sets:

The formula for actual calculation:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

We can see how this relates back to the law of total probability - multiply both sides by $P(B)$ and you get $P(A|B)P(B) = P(A \cap B)$ - replaced back into the law of total probability we get $P(A) = \sum_n P(A \cap B_n)$.

This may not seem like an improvement at first, but try to relate it back to the above picture - if you think of sets as physical objects, we're saying that the total probability of $A$ given $B$ is all the little pieces of it intersected with $B$, added together. The conditional probability is then just that again, but divided by the probability of $B$ itself happening in the first place.

### Bayes Theorem

Here is is, the seemingly magic tool:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In words - the probability of $A$ conditioned on $B$ is the probability of $B$ conditioned on $A$, times the probability of $A$ and divided by the probability of $B$. These unconditioned probabilities are referred to as "prior beliefs", and the conditioned probabilities as "updated."

Why is this important? Scroll back up to the XKCD example - the Bayesian statistician draws a less absurd conclusion because their prior belief in the likelihood that the sun will go nova is extremely low. So, even when updated based on evidence from a detector that is $35/36 = 0.972$ accurate, the prior belief doesn't shift enough to change their overall opinion.

There's many examples of Bayes' theorem - one less absurd example is to apply to [breathalyzer tests](https://www.bayestheorem.net/breathalyzer-example/). You may think that a breathalyzer test that is 100% accurate for true positives (detecting somebody who is drunk) is pretty good, but what if it also has 8% false positives (indicating somebody is drunk when they're not)? And furthermore, the rate of drunk driving (and thus our prior belief)  is 1/1000.

What is the likelihood somebody really is drunk if they test positive? Some may guess it's 92% - the difference between the true positives and the false positives. But we have a prior belief of the background/true rate of drunk driving. Sounds like a job for Bayes' theorem!

$$
\begin{aligned}
P(Drunk | Positive) &= \frac{P(Positive | Drunk)P(Drunk)}{P(Positive)} \\
&= \frac{1 \times 0.001}{0.08} \\
&= 0.0125
\end{aligned}
$$

In other words, the likelihood that somebody is drunk given they tested positive with a breathalyzer in this situation is only 1.25% - probably much lower than you'd guess. This is why, in practice, it's important to have a repeated test to confirm (the probability of two false positives in a row is $0.08 * 0.08 = 0.0064$, much lower), and Bayes' theorem has been relevant in court cases where proper consideration of evidence was important.

## Live Lecture - Deriving Bayes' Theorem, Calculating Bayesian Confidence

Notice that $P(A|B)$ appears in the above laws - in Bayesian terms, this is the belief in $A$ updated for the evidence $B$. So all we need to do is solve for this term to derive Bayes' theorem. Let's do it together!

In [1]:
# Activity 2 - Use SciPy to calculate Bayesian confidence intervals
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bayes_mvs.html#scipy.stats.bayes_mvs

from scipy import stats
import numpy as np

coinflips = np.random.binomial(n=1, p=0.5, size=100)
print(coinflips)

[1 1 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1
 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1
 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 1]


In [2]:
# Frequentist approach (from yesterday)
def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
  return (mean, mean - interval, mean + interval)

confidence_interval(coinflips)

(0.48, 0.38036914695852936, 0.5796308530414707)

In [3]:
import pandas as pd
pd.DataFrame(coinflips).describe()

0
count  100.000000
mean     0.480000
std      0.502117
min      0.000000
25%      0.000000
50%      0.000000
75%      1.000000
max      1.000000

In [4]:
stats.bayes_mvs(coinflips)

(Mean(statistic=0.48, minmax=(0.3966289819625553, 0.5633710180374446)),
 Variance(statistic=0.2573195876288659, minmax=(0.20255593542955683, 0.32396091281978656)),
 Std_dev(statistic=0.5059610993316946, minmax=(0.4500621461860093, 0.5691756432067226)))

In [5]:
# Let's do something else medical
import random

# We have two groups of people, one treated one non-treated
# Treated people recover with probability 0.65
# Non-treated people recover with probability 0.4
treatment_group = np.random.binomial(n=1, p=0.65, size=40)
nontreated_group = np.random.binomial(n=1, p=0.4, size=40)

print(treatment_group)

[1 0 1 1 0 1 0 0 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1]


In [6]:
import pandas as pd
df = pd.DataFrame({'treated': treatment_group,
                   'untreated': nontreated_group})
df.describe()

treated  untreated
count  40.000000  40.000000
mean    0.750000   0.400000
std     0.438529   0.496139
min     0.000000   0.000000
25%     0.750000   0.000000
50%     1.000000   0.000000
75%     1.000000   1.000000
max     1.000000   1.000000

In [7]:
df.head()

treated  untreated
0        1          0
1        0          1
2        1          0
3        1          1
4        0          1

In [8]:
# Frequentist hypothesis test
from scipy import stats
stats.ttest_ind(df.treated, df.untreated)

Ttest_indResult(statistic=3.3429685305864343, pvalue=0.0012752807646138114)

In [9]:
stats.bayes_mvs(df.treated)

(Mean(statistic=0.75, minmax=(0.6331749300749732, 0.8668250699250267)),
 Variance(statistic=0.20270270270270271, minmax=(0.13743254230208907, 0.2918811461266654)),
 Std_dev(statistic=0.447193728602567, minmax=(0.37071895325446885, 0.5402602577708872)))

In [10]:
stats.bayes_mvs(df.untreated)

(Mean(statistic=0.4, minmax=(0.2678275213494736, 0.5321724786505264)),
 Variance(statistic=0.25945945945945953, minmax=(0.175913654146674, 0.3736078670421317)),
 Std_dev(statistic=0.5059419487983547, minmax=(0.4194206172169819, 0.6112347070006183)))

## Assignment - Code it up!

Most of the above was pure math - write Python code to reproduce the results. This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up, and as a stretch goal - refactor your code into helpful reusable functions!

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/) - you could and should create something similar!

Stretch goal - apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective.

In [20]:
import numpy as np
import pandas as pd
import random
from scipy import stats

# create the data
# cars should go faster (yes = 1, no = 0) has a probability of 0.8
# cars should be safer (yes = 1, no = 0) has a probability of 0.45

faster = np.random.binomial(n = 1, p = 0.8, size = 125)
safer = np.random.binomial(n = 1, p = 0.45, size = 125)

print(faster)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 0 1 0 0 1 1 1 1 1 1 0]


In [24]:
def confidence_interval(data, confidence = 0.95):
    data = np.array(data)  # data in list form
    mean = np.mean(data)
    n = len(data)  # number of entries in the list
    standard_error = stats.sem(data)
    interval = standard_error * stats.t.ppf((1 + confidence) / 2.0, n - 1)  # ppf is probability density function
    return (mean, mean - interval, mean + interval)

confidence_interval(faster)

(0.856, 0.7935956954501098, 0.9184043045498902)

In [25]:
confidence_interval(safer)

(0.432, 0.34395342623628106, 0.520046573763719)

In [12]:
df = pd.DataFrame({'Faster' : faster, 'Safer' : safer})
df.head()

Faster  Safer
0       1      0
1       1      0
2       0      1
3       0      1
4       1      1

In [13]:
df.describe()

Faster       Safer
count  125.00000  125.000000
mean     0.78400    0.392000
std      0.41317    0.490161
min      0.00000    0.000000
25%      1.00000    0.000000
50%      1.00000    0.000000
75%      1.00000    1.000000
max      1.00000    1.000000

In [19]:
from scipy import stats
stats.ttest_ind(df.Faster, df.Safer)

Ttest_indResult(statistic=6.836553337644748, pvalue=6.269378367590377e-11)

In [16]:
stats.bayes_mvs(df.Faster)

(Mean(statistic=0.784, minmax=(0.722756760724396, 0.845243239275604)),
 Variance(statistic=0.1735081967213115, minmax=(0.14019524411626888, 0.21320949754843296)),
 Std_dev(statistic=0.41569046256281783, minmax=(0.37442655370081446, 0.46174613972228606)))

In [17]:
stats.bayes_mvs(df.Safer)

(Mean(statistic=0.392, minmax=(0.31934457131627764, 0.4646554286837224)),
 Variance(statistic=0.24419672131147543, minmax=(0.19731182505252656, 0.30007262617927605)),
 Std_dev(statistic=0.49315106637847755, minmax=(0.44419795705577775, 0.5477888518209146)))